In [1]:
import saspy
sas = saspy.SASsession(cfgname='oda', )

SAS Connection established. Subprocess id is 15840



In [2]:
from IPython.display import display, HTML

In [3]:
sd = sas.sasdata("cars", libref= "sashelp")
print(sd.obs())
sd.head()


428


,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945.0,33337.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,21761.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,24647.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,30299.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,39014.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [4]:
res = sas.submit(
'''
PROC sql;
create table df as
select
    make,
    model,
    drivetrain,
    count(*) as count
from
    sashelp.cars
group by
    1,2,3
order by count desc
'''
)
sas.sasdata('df').head()

,Make,Model,DriveTrain,count
0,Toyota,Corolla CE 4dr,Front,1.0
1,Pontiac,Aztekt,Front,1.0
2,Jaguar,S-Type R 4dr,Rear,1.0
3,Lexus,IS 300 4dr manual,Rear,1.0
4,Mitsubishi,Eclipse GTS 2dr,Front,1.0


In [5]:
res = sas.submit(
'''
proc sql;
select * from sashelp.cars
where 
    make = 'Mercedes-Benz'
    and model = " C240 4dr"
;
quit;
'''
)
HTML(res['LST'])

Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,Engine Size (L),Cylinders,Horsepower,MPG (City),MPG (Highway),Weight (LBS),Wheelbase (IN),Length (IN)
Mercedes-Benz,C240 4dr,Sedan,Europe,Rear,"$32,280","$30,071",2.6,6,168,20,25,3360,107,178
Mercedes-Benz,C240 4dr,Sedan,Europe,All,"$33,480","$31,187",2.6,6,168,19,25,3360,107,178


https://support.sas.com/documentation/cdl/en/proc/61895/HTML/default/viewer.htm#a000063661.htm

https://support.sas.com/resources/papers/proceedings09/060-2009.pdf

In [6]:
sas.submitLOG(
'''
PROC SORT DATA=sashelp.cars OUT=cars_sorted;
    BY make model driveTrain;
RUN;

PROC TRANSPOSE DATA=cars_sorted OUT=cars_transposed (rename=(COL1=prices)) name=price_type;
    BY make model driveTrain;
    VAR msrp invoice;
RUN;
'''
)
df = sas.sasdata('cars_transposed').to_df()
df.head()


47                                                         The SAS System                          Tuesday, May  7, 2024 01:32:00 PM

303        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
303      !  graphics on / outputfmt=png;
304        
305        
306        PROC SORT DATA=sashelp.cars OUT=cars_sorted;
307            BY make model driveTrain;
308        RUN;
309        
310        PROC TRANSPOSE DATA=cars_sorted OUT=cars_transposed (rename=(COL1=prices)) name=price_type;
311            BY make model driveTrain;
312            VAR msrp invoice;
313        RUN;
314        
315        
316        
317        ods html5 (id=saspy_internal) close;ods listing;
318        

48                                                         The SAS System                          Tuesday, May  7, 2024 01:32:00 PM

319        


,Make,Model,DriveTrain,price_type,prices
0,Acura,3.5 RL 4dr,Front,MSRP,43755.0
1,Acura,3.5 RL 4dr,Front,Invoice,39014.0
2,Acura,3.5 RL w/Navigation 4dr,Front,MSRP,46100.0
3,Acura,3.5 RL w/Navigation 4dr,Front,Invoice,41100.0
4,Acura,MDX,All,MSRP,36945.0


In [7]:
sas.submit("""
data fishdata;
   infile datalines missover;
   input Location & $10. Date date7.
         Length1 Weight1 Length2 Weight2 Length3 Weight3
         Length4 Weight4;
   format date date7.;
   datalines;
Cole Pond   1JAN24 31 .25 32 .3  32 .25 33 .3
Cole Pond   1FEB24 33 .32 34 .41 37 .48 32 .28
Cole Pond   1MAR24 29 .23 30 .25 34 .47 32 .3
Eagle Lake  1JAN24 32 .35 32 .25 33 .30
Eagle Lake  1FEB24 30 .20 36 .45
Eagle Lake  1MAR24 33 .30 33 .28 34 .42
;
""")
df = sas.sasdata('fishdata').to_df()
df

,Location,Date,Length1,Weight1,Length2,Weight2,Length3,Weight3,Length4,Weight4
0,Cole Pond,2024-01-01,31.0,0.25,32.0,0.30,32.0,0.25,33.0,0.30
1,Cole Pond,2024-02-01,33.0,0.32,34.0,0.41,37.0,0.48,32.0,0.28
2,Cole Pond,2024-03-01,29.0,0.23,30.0,0.25,34.0,0.47,32.0,0.30
3,Eagle Lake,2024-01-01,32.0,0.35,32.0,0.25,33.0,0.30,NaN,NaN
4,Eagle Lake,2024-02-01,30.0,0.20,36.0,0.45,NaN,NaN,NaN,NaN
5,Eagle Lake,2024-03-01,33.0,0.30,33.0,0.28,34.0,0.42,NaN,NaN


In [8]:
sas.submit("""
proc transpose data=fishdata
     out=fishlength(rename=(col1=Measurement));
   var length1-length4;
   by location date;
run;
""")

sas.sasdata('fishlength').to_df()


,Location,Date,_NAME_,Measurement
0,Cole Pond,2024-01-01,Length1,31.0
1,Cole Pond,2024-01-01,Length2,32.0
2,Cole Pond,2024-01-01,Length3,32.0
3,Cole Pond,2024-01-01,Length4,33.0
4,Cole Pond,2024-02-01,Length1,33.0
5,Cole Pond,2024-02-01,Length2,34.0
6,Cole Pond,2024-02-01,Length3,37.0
7,Cole Pond,2024-02-01,Length4,32.0
8,Cole Pond,2024-03-01,Length1,29.0
9,Cole Pond,2024-03-01,Length2,30.0


In [9]:
# do same using pandas
df.set_index(['Location', 'Date']).stack().reset_index().rename(columns={'level_2': 'Measurement', 0: 'Value'})

,Location,Date,Measurement,Value
0,Cole Pond,2024-01-01,Length1,31.00
1,Cole Pond,2024-01-01,Weight1,0.25
2,Cole Pond,2024-01-01,Length2,32.00
3,Cole Pond,2024-01-01,Weight2,0.30
4,Cole Pond,2024-01-01,Length3,32.00
5,Cole Pond,2024-01-01,Weight3,0.25
6,Cole Pond,2024-01-01,Length4,33.00
7,Cole Pond,2024-01-01,Weight4,0.30
8,Cole Pond,2024-02-01,Length1,33.00
9,Cole Pond,2024-02-01,Weight1,0.32
